In [1]:
import numpy as np
import os
import pandas as pd


In [2]:
import sqlite3 as sql

In [3]:
dirWork         = os.getcwd()
dirInput        = os.path.join(dirWork, r'input'       )
dirIntermediate = os.path.join(dirWork, r'intermediate')
dirResults      = os.path.join(dirWork, r'results'     )

print("Working Directory: "      + dirWork        )
print("Input Directory: "        + dirInput       )
print("Intermediate Directory: " + dirIntermediate)
print("Results Directory: "      + dirResults     )

Working Directory: e:\GitHub\UDOT-Segment-Factors
Input Directory: e:\GitHub\UDOT-Segment-Factors\input
Intermediate Directory: e:\GitHub\UDOT-Segment-Factors\intermediate
Results Directory: e:\GitHub\UDOT-Segment-Factors\results


In [4]:
df_CCSHourDirLane_Avenue = pd.read_csv(os.path.join(dirIntermediate,'CCSHourDirLane_Avenue.csv'))
display(df_CCSHourDirLane_Avenue)

df_CCSHourDirLane_OldFormat = pd.read_csv(os.path.join(dirIntermediate,'CCSHourDirLane_2019Aug.csv'))
display(df_CCSHourDirLane_OldFormat)

df_CCSHourDirLane_NewFormat = pd.read_csv(os.path.join(dirIntermediate,'CCSHourDirLane_NewFormat.csv'))
display(df_CCSHourDirLane_NewFormat)

,STATION,DIR,YEAR,MONTH,DAY,LANE,HOUR,HOURVOL,DOW
0,301,N,2013,1,1,0,0,358,1
1,301,N,2013,1,1,0,1,435,1
2,301,N,2013,1,1,0,2,277,1
3,301,N,2013,1,1,0,3,163,1
4,301,N,2013,1,1,0,4,129,1
...,...,...,...,...,...,...,...,...,...
9076141,714,P,2017,12,31,0,19,34,6
9076142,714,P,2017,12,31,0,20,23,6
9076143,714,P,2017,12,31,0,21,32,6
9076144,714,P,2017,12,31,0,22,30,6


,STATION,DIR,YEAR,MONTH,DAY,LANE,HOUR,HOURVOL,DOW
0,301,N,2019,8,11,1,0,511,6
1,301,N,2019,8,11,1,1,284,6
2,301,N,2019,8,11,1,2,188,6
3,301,N,2019,8,11,1,3,140,6
4,301,N,2019,8,11,1,4,147,6
...,...,...,...,...,...,...,...,...,...
341106,714,P,2019,8,31,1,19,48,5
341107,714,P,2019,8,31,1,20,36,5
341108,714,P,2019,8,31,1,21,23,5
341109,714,P,2019,8,31,1,22,23,5


,STATION,DIR,YEAR,MONTH,DAY,LANE,HOUR,HOURVOL,DOW
0,301,N,2018,1,1,1,0,155,0
1,301,N,2018,1,1,2,0,238,0
2,301,N,2018,1,1,3,0,82,0
3,301,P,2018,1,1,1,0,80,0
4,301,P,2018,1,1,2,0,167,0
...,...,...,...,...,...,...,...,...,...
18883064,733,N,2022,9,30,3,0,162,4
18883065,733,P,2022,9,30,1,0,8,4
18883066,733,P,2022,9,30,2,0,114,4
18883067,733,P,2022,9,30,3,0,165,4


In [5]:
# initialize dataframe with avenue data
df_CCSHourDirLane = df_CCSHourDirLane_Avenue.copy()

#add 2019 Aug and no august data
df_CCSHourDirLane = df_CCSHourDirLane.append(df_CCSHourDirLane_OldFormat, ignore_index=True)
df_CCSHourDirLane = df_CCSHourDirLane.append(df_CCSHourDirLane_NewFormat, ignore_index=True)

df_CCSHourDirLane

,STATION,DIR,YEAR,MONTH,DAY,LANE,HOUR,HOURVOL,DOW
0,301,N,2013,1,1,0,0,358,1
1,301,N,2013,1,1,0,1,435,1
2,301,N,2013,1,1,0,2,277,1
3,301,N,2013,1,1,0,3,163,1
4,301,N,2013,1,1,0,4,129,1
...,...,...,...,...,...,...,...,...,...
28300321,733,N,2022,9,30,3,0,162,4
28300322,733,P,2022,9,30,1,0,8,4
28300323,733,P,2022,9,30,2,0,114,4
28300324,733,P,2022,9,30,3,0,165,4


In [6]:
df_CCSHourDirLane.set_index(['STATION','DIR','YEAR','MONTH','DAY','LANE','HOUR'])

HOURVOL  DOW
STATION DIR YEAR MONTH DAY LANE HOUR              
301     N   2013 1     1   0    0         358    1
                                1         435    1
                                2         277    1
                                3         163    1
                                4         129    1
...                                       ...  ...
733     N   2022 9     30  3    0         162    4
        P   2022 9     30  1    0           8    4
                           2    0         114    4
                           3    0         165    4
                           4    0          80    4

[28300326 rows x 2 columns]

In [7]:
df_CCSHourDir = df_CCSHourDirLane.groupby(['STATION','YEAR','MONTH','DAY','DOW','HOUR','DIR'],as_index=False).agg({'HOURVOL':[np.sum]})
df_CCSHourDir = df_CCSHourDir.droplevel(1, axis=1)
df_CCSHourDir

,STATION,YEAR,MONTH,DAY,DOW,HOUR,DIR,HOURVOL
0,301,2013,1,1,1,0,N,358
1,301,2013,1,1,1,0,P,288
2,301,2013,1,1,1,1,N,435
3,301,2013,1,1,1,1,P,292
4,301,2013,1,1,1,2,N,277
...,...,...,...,...,...,...,...,...
9563592,733,2022,9,28,2,0,P,29398
9563593,733,2022,9,29,3,0,N,37108
9563594,733,2022,9,29,3,0,P,32318
9563595,733,2022,9,30,4,0,N,44872


In [8]:
df_CCSHourDir.dtypes

STATION     int64
YEAR        int64
MONTH       int64
DAY         int64
DOW         int64
HOUR        int64
DIR        object
HOURVOL     int64
dtype: object

In [11]:
#for checks and understanding
#calculate number of days with data per station per month
df_1 = df_CCSHourDir.groupby(['STATION','MONTH','YEAR','DAY'],as_index=False).agg(NUMHOURS=('HOURVOL','size'))
df_2 = df_1.groupby(['STATION','YEAR','MONTH'],as_index=False).agg(NUMDAYS=('DAY','size'))
df_3 = df_2.pivot_table(index=['STATION'], columns=('YEAR','MONTH'), values='NUMDAYS').reset_index()

df_3 = df_3.fillna(0)
df_3 = df_3.astype(np.int32)

#export
df_3.to_csv(os.path.join(dirIntermediate, r'StationMonthDays.csv'),index=False)

In [12]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_3)
pd.set_option('display.max_rows', 10)

YEAR  STATION 2013                                             2014          \
MONTH            1   2   3   4   5   6   7   8   9  10  11  12    1   2   3   
0         301   31  28  30  30  31  28  28  31  30  22  30  31   31  28  30   
1         302   31  28  30  30  31  30  31  29  30  31  30  31   31  28  30   
2         303   23  27  30  30  31  30  31  31  30  31  30  31   31  28  30   
3         304   31  28  30  30  31  30  31  31  30  31  30  31   31  28  30   
4         305   28  28  30  30  28  30  25  31  30  31  30  31   31  22  30   
5         306   30  25  30  30  31  30  31  31  30  31  30  31   31  28  30   
6         307   31  28  30  30  31  30  31  31  30  31  30  27   31  20  30   
7         308   31  28  30  30  31  30  30  31  30  31  29  28   31  28  30   
8         309   25  28  30  30  31  30  30  31  30  31  30  27   31  28  29   
9         310   31  27  30  30  30  22  31  31  30  31  30  31   31  28  29   
10        312   23  28  28  30  18  30  31  31  30  31  30  31   30  28  29   
11        313   31  28  30  30  31  30  31  31  30  31  30  29   31  28  29   
12        314   30  28  30  30  31  30  31  30  30  31  11  31   31  28  30   
13        315   30  28  29  30  31  29  31  31  30  31  30  31   31  28  30   
14        316   31  28  30  30  31  30  31  31  30  31  30  31   31  28  30   
15        317   31  28  30  30  31  30  31  31  30  31  30  29   31  28  30   
16        318   27  28  30  30  31  30  31  31  30  31  30  26   31  28  29   
17        319   31  28  29  30  31  30  31  31  30  31  30  31   31  28  30   
18        320   24  28  30  30  22  18  27  30  26  29  30  28   31  25  30   
19        321   31  28  30  30  31  30  31  31  30  31  30  31   31  28  29   
20        322   31  28  30  30  31  30  27  30  30  25  30  31   31  28  30   
21        323   31  28  30  30  31  30  31  31  30  31  30  31   31  28  30   
22        324   31  28  30  30  31  30  31  31  30  31  30  31   31  28  30   
23        325   31  28  30  30  30  30  31  31  30  31  30  31   31  28  30   
24        327   31  28  29  30  31  30  31  31  30  31  30  31   30  28  30   
25        329   22  28  30  30  31  30  31  31  30  31  30  25   29  24  29   
26        332   31  25  30  30  22  30  31  31  30  15  23  27   19  28  26   
27        333   31  28  30  30  31  30  31  31  29  31  30  30   30  28  30   
28        335   31  28  30  30  31  30  29  28  30  27  30  29   30  28  30   
29        340   31  26  28  30  31  30  31  31  30  31  30  31   31  28  30   
30        341   31  28  30  30  31  29  30  31  30  31  30  31   31  28  29   
31        348   23  28  30  30  31  30  31  31  30  31  28  25   31  28  30   
32        349   31  28  30  30  30  30  31  31  30  31  29  31   31  28  30   
33        350   26  28  30  30  31  30  31  31  30  31  30  29   31  28  29   
34        351   31  28  30  30  31  30  31  31  30  31  30  31   31  28  30   
35        353   31  28  30  30  30  30  31  31  30  31  30  30   31  28  29   
36        354   31  28  30  30  31  30  31  31  29  31  30  30   31  28  30   
37        355   31  28  30  30  31  30  31  31  30  31  30  31   31  18  30   
38        362   31  28  30  30  31  30  31  31  30  31  30  31   31  28  30   
39        363   31  28  30  30  31  30  31  27  30  31  30  31   31  28  29   
40        382   31  28  30  30  31  30  31  31  26  31  30  31   31  24  26   
41        400   31  26  29  30  30  30  31  31  28  29  29  31   31  28  27   
42        401   31  28  30  30  31  30  31  31  30  28  28  25   12  27  29   
43        402   31  28  30  30  31  30  31  31  29  30  26  27   29  24  29   
44        403   31  27  30  27  31  30  31  31  26  31  29  30   31  28  28   
45        404   31  28  30  30  31  30  31  27  27  31  30  31   31  28  31   
46        405   31  28  30  30  31  30  31  31  30  31  30  31   31  23  21   
47        406   31  28  30  30  31  30  31  31  30  31  30  31   31  28  29   
48        407   31  28  30  25  31  30  31  31  30

In [14]:
#export to csv
df_CCSHourDir.to_csv(os.path.join(dirIntermediate, r'CCSHourDir_Combined_2013to2022.csv'),index=False)

In [15]:
#combine truck data
dfTruckDataAvenue = pd.read_csv(os.path.join(dirIntermediate,'AnnualTruckDailyTraffic_Avenue.csv'))
display(dfTruckDataAvenue)

dfTruckDataNewFormat = pd.read_csv(os.path.join(dirIntermediate,'AnnualTruckDailyTraffic_NewFormat.csv'))
display(dfTruckDataNewFormat)

# initialize dataframe with avenue data
dfTruckData = dfTruckDataAvenue.copy()

#add 2019 Aug and no august data
dfTruckData = dfTruckData.append(dfTruckDataNewFormat, ignore_index=True)

dfTruckData

,YEAR,ROUTE,BEGMP,ENDMP,DESC,AADT,SUTRK,CUTRK,SegID
0,2012,0006,0.00,46.01,Nevada State Line via SR 6 - Antelope Springs,325.0,0.24,0.23,000600000
1,2012,0006,46.01,77.54,Antelope Springs via SR 6 - Gunnison Massacre ...,340.0,0.17,0.33,000604602
2,2012,0006,77.54,82.88,Gunnison Massacre Site via SR 6 - Main St Hin...,420.0,0.16,0.26,000607754
3,2012,0006,82.88,83.89,Main St via SR 6 (500 N) Hinckley - SR 257 (40...,1570.0,0.14,0.19,000608288
4,2012,0006,83.89,87.68,SR 257 (4000 W) Hinckley via SR 6 - 1000 W to ...,3270.0,0.13,0.12,000608390
...,...,...,...,...,...,...,...,...,...
9321,2016,0320,0.00,2.19,Emergency Vehicle Operation Facility Camp Will...,160.0,0.10,0.29,032000000
9322,2016,0491,0.00,0.44,SR 191 (Main St) via Center St - 500 E Monticello,5400.0,0.05,0.24,049100000
9323,2016,0491,0.44,2.01,500 E via Center St - Port of Entry Monticello,3000.0,0.05,0.24,049100044
9324,2016,0491,2.01,17.02,Monticello POE via SR 491 - Colorado State Lin...,2800.0,0.05,0.24,049100202


,YEAR,ROUTE,BEGMP,ENDMP,DESC,AADT,SUTRK,CUTRK,SegID


,YEAR,ROUTE,BEGMP,ENDMP,DESC,AADT,SUTRK,CUTRK,SegID
0,2012,0006,0.00,46.01,Nevada State Line via SR 6 - Antelope Springs,325.0,0.24,0.23,000600000
1,2012,0006,46.01,77.54,Antelope Springs via SR 6 - Gunnison Massacre ...,340.0,0.17,0.33,000604602
2,2012,0006,77.54,82.88,Gunnison Massacre Site via SR 6 - Main St Hin...,420.0,0.16,0.26,000607754
3,2012,0006,82.88,83.89,Main St via SR 6 (500 N) Hinckley - SR 257 (40...,1570.0,0.14,0.19,000608288
4,2012,0006,83.89,87.68,SR 257 (4000 W) Hinckley via SR 6 - 1000 W to ...,3270.0,0.13,0.12,000608390
...,...,...,...,...,...,...,...,...,...
9321,2016,0320,0.00,2.19,Emergency Vehicle Operation Facility Camp Will...,160.0,0.10,0.29,032000000
9322,2016,0491,0.00,0.44,SR 191 (Main St) via Center St - 500 E Monticello,5400.0,0.05,0.24,049100000
9323,2016,0491,0.44,2.01,500 E via Center St - Port of Entry Monticello,3000.0,0.05,0.24,049100044
9324,2016,0491,2.01,17.02,Monticello POE via SR 491 - Colorado State Lin...,2800.0,0.05,0.24,049100202


In [16]:
import csv
dfTruckData.to_csv(os.path.join(dirIntermediate, r'TruckData_Combined_2012to2019.csv'),index=False,quotechar='"', quoting=csv.QUOTE_NONNUMERIC)

In [17]:
dfTruckData[(dfTruckData['ROUTE']=='0175') & (dfTruckData['YEAR']==2019)]

,YEAR,ROUTE,BEGMP,ENDMP,DESC,AADT,SUTRK,CUTRK,SegID
